In [2]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')


True

This implementation showcases how you could build AGoT using DSPy's composable abstractions. Let me explain the key components:
Core Components of the Implementation

1. DSPy Signatures and Modules for each AGoT Operation,

- `T0`: Generates initial thoughts (first layer)
- `Te`: Expands the graph with additional thoughts and edges
- `C`: Checks thought complexity to determine if recursion is needed
- `Eval`: Evaluates non-complex thoughts
- `Phi`: Synthesizes the final answer from the complete graph


2. Graph Structure: I use NetworkX to represent the directed acyclic graph (DAG) structure, with each node containing a Thought object that tracks important metadata.

3. Recursive Implementation: The _recursive_agot method implements the core algorithm described in the paper, including:

- Layer-by-layer generation of thoughts
- Complexity checking
- Recursive processing for complex thoughts
- Final answer synthesis


5. Visualization: The implementation includes a `visualize_graph` method to help understand the resulting reasoning structure.

### How This Maps to the Paper
The implementation closely follows the mathematical formalism in Section 2 of the paper:

- The heritage tracking system `(h = (s₀, s₁, ..., sₙ)) `is implemented using a list of layer and node index tuples
- The complexity function `C: T × G → {0, 1}` maps directly to the ComplexityChecker module
- The evaluation function `Eval: T × G → A` corresponds to the ThoughtEvaluator module
- The final synthesis function `Φ: G → T` is implemented as the FinalAnswerSynthesizer module

Implementation Notes

DSPy makes it easy to swap different language models as backends, which would be useful for testing different LLMs with AGoT.
The implementation focuses on the core AGoT algorithm rather than specific optimization techniques mentioned in the paper.
In a full implementation, you might want to add:

Better prompt engineering for each module
More sophisticated termination conditions
Parallelization for asynchronous thought evaluation
Metrics collection and evaluation against benchmarks

In [32]:
import dspy
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass

# Define the signature for the initial thoughts generation
class InitialThoughtsGenerator(dspy.Signature):
    """Generate initial thoughts for decomposing a complex problem."""
    query = dspy.InputField(desc="The original query or problem to solve")
    max_thoughts = dspy.InputField(desc="Maximum number of initial thoughts to generate")
    
    thoughts = dspy.OutputField(desc="A list of complete, meaningful thoughts to tackle the problem. Each thought should be a fully formed idea, not individual characters or fragments.")
    strategy = dspy.OutputField(desc="A strategy to guide the exploration of these thoughts")

# Define the signature for generating subsequent thoughts
class ThoughtExpander(dspy.Signature):
    """Generate additional thoughts based on the current graph state."""
    query = dspy.InputField(desc="The original query or problem to solve")
    current_graph = dspy.InputField(desc="Current state of the graph including all thoughts and their answers")
    max_new_thoughts = dspy.InputField(desc="Maximum number of new thoughts to generate")
    
    new_thoughts = dspy.OutputField(desc="A list of new complete, meaningful thoughts to add to the graph. Each thought should be a fully formed idea, not individual characters or fragments.")
    new_edges = dspy.OutputField(desc="A list of edges to add, specified as [(source_id, target_id), ...] tuples")
    strategy = dspy.OutputField(desc="A strategy to guide the exploration of these thoughts")

# Define a signature for complexity checking
class ComplexityChecker(dspy.Signature):
    """Determine if a thought is complex enough to warrant further decomposition."""
    thought = dspy.InputField(desc="The thought content to evaluate")
    graph_context = dspy.InputField(desc="Current state of the graph for context")
    
    is_complex = dspy.OutputField(desc="1 if the thought is complex and should be decomposed, 0 otherwise")
    reason = dspy.OutputField(desc="Reasoning for the complexity determination")

# Define a signature for thought evaluation
class ThoughtEvaluator(dspy.Signature):
    """Evaluate a thought to produce an answer or response."""
    thought = dspy.InputField(desc="The thought to evaluate")
    graph_context = dspy.InputField(desc="Current graph context to inform the evaluation")
    
    answer = dspy.OutputField(desc="Answer or response to the thought")

# Define a signature for final answer synthesis
class FinalAnswerSynthesizer(dspy.Signature):
    """Synthesize a final answer from the complete graph of thoughts."""
    query = dspy.InputField(desc="The original query or problem")
    graph = dspy.InputField(desc="The complete graph with all thoughts and answers")
    
    final_answer = dspy.OutputField(desc="The final answer to the original query")

# Create DSPy modules for each signature
class T0(dspy.Module):
    """Module for generating initial thoughts (T₀ in the paper)."""
    def __init__(self):
        super().__init__()
        self.generator = dspy.ChainOfThought(InitialThoughtsGenerator)
    
    def forward(self, query, max_thoughts=3):
        # Ensure the query is treated as a complete prompt, not split
        return self.generator(query=query, max_thoughts=max_thoughts)

class Te(dspy.Module):
    """Module for expanding thoughts with new nodes and edges (Tₑ in the paper)."""
    def __init__(self):
        super().__init__()
        self.expander = dspy.ChainOfThought(ThoughtExpander)
    
    def forward(self, query, current_graph, max_new_thoughts=3):
        return self.expander(
            query=query, 
            current_graph=current_graph,
            max_new_thoughts=max_new_thoughts
        )

class C(dspy.Module):
    """Module for complexity checking (C in the paper)."""
    def __init__(self):
        super().__init__()
        self.checker = dspy.Predict(ComplexityChecker)
    
    def forward(self, thought, graph_context):
        result = self.checker(thought=thought, graph_context=graph_context)
        return int(result.is_complex), result.reason

class Eval(dspy.Module):
    """Module for thought evaluation (Eval in the paper)."""
    def __init__(self):
        super().__init__()
        self.evaluator = dspy.Predict(ThoughtEvaluator)
    
    def forward(self, thought, graph_context):
        return self.evaluator(thought=thought, graph_context=graph_context).answer

class Phi(dspy.Module):
    """Module for final answer synthesis (Φ in the paper)."""
    def __init__(self):
        super().__init__()
        self.synthesizer = dspy.Predict(FinalAnswerSynthesizer)
    
    def forward(self, query, graph):
        return self.synthesizer(query=query, graph=graph).final_answer

# Define a data structure for thoughts
@dataclass
class Thought:
    id: str
    content: str
    layer: int
    strategy: str
    answer: Optional[str] = None
    is_complex: bool = False
    nested_graph: Optional[Any] = None
    
    def __str__(self):
        status = "COMPLEX" if self.is_complex else "EVALUATED"
        if self.answer:
            status = f"{status}: {self.answer[:50]}..." if len(self.answer) > 50 else f"{status}: {self.answer}"
        return f"Thought {self.id} [Layer {self.layer}]: {self.content[:50]}... ({status})"

# The main AGoT implementation
class AdaptiveGraphOfThoughts(dspy.Module):
    def __init__(self, lm=None, max_depth=1, max_layers=3, max_nodes=3):
        super().__init__()
        
        # Set language model
        if lm:
            dspy.settings.configure(lm=lm)
        
        # Initialize the DSPy modules
        self.t0 = T0()
        self.te = Te()
        self.c = C()
        self.eval = Eval()
        self.phi = Phi()
        
        # Set limits
        self.max_depth = max_depth
        self.max_layers = max_layers
        self.max_nodes = max_nodes
    
    def _format_graph_for_context(self, G):
        """Format the graph state as a string for context in prompts."""
        result = []
        for node_id in nx.topological_sort(G):
            thought = G.nodes[node_id]['data']
            result.append(str(thought))
            
            # Add edge information
            for successor in G.successors(node_id):
                result.append(f"  ↳ connects to Thought {successor}")
                
        return "\n".join(result)
    
    def _is_edge_safe(self, graph, source, target):
        """
        Check if adding an edge would create a cycle in the graph.
        
        Args:
            G (nx.DiGraph): The current directed graph
            source (str): Source node ID
            target (str): Target node ID
        
        Returns:
            bool: True if adding the edge would not create a cycle, False otherwise
        """
        # Create a copy of the graph to test edge addition
        test_graph = graph.copy()
        test_graph.add_edge(source, target)
        
        try:
            # Check if there are any cycles in the graph
            nx.find_cycle(test_graph)
            print('[DEBUG] Cycles found')
            return False  # Cycle found, edge is not safe
        except nx.NetworkXNoCycle:
            print('[DEBUG] Cycles not found')
            return True  # No cycle, edge i
    
    def _recursive_agot(self, query, heritage, parent_graph=None, depth=0):
        """Recursive implementation of the AGoT algorithm."""
        # Initialize graph for this recursion level
        G = nx.DiGraph()
        
        # Process layer by layer
        for layer in range(self.max_layers):
            # Generate thoughts for this layer
            if layer == 0 and depth == 0:
                # Generate initial thoughts for the top-level graph
                # Add explicit instructions to handle multilingual queries properly
                enhanced_query = f"""
                IMPORTANT: Please process this query in its original language.
                DO NOT split the query into individual characters.
                Generate {self.max_nodes} complete, meaningful thoughts related to this query.
                Query: {query}
                """
                
                print(f"\n[DEBUG] Depth {depth}, Layer {layer}: Generating initial thoughts for query: {query}")
                result = self.t0(enhanced_query, self.max_nodes)
                
                # Extract thoughts - ensure we have proper complete thoughts
                thoughts = result.thoughts
                if not thoughts or isinstance(thoughts, str):
                    # Fallback if the output format is incorrect
                    if isinstance(thoughts, str):
                        # Try to parse if it's a string
                        import re
                        thoughts = re.split(r'\d+\.\s+', thoughts)
                        thoughts = [t.strip() for t in thoughts if t.strip()]
                    
                    # If still empty, create default thoughts
                    if not thoughts:
                        thoughts = [
                            f"Analyze key aspects of {query}",
                            f"Identify main components in {query}",
                            f"Determine methodological approach for {query}"
                        ]
                
                print(f"[DEBUG] Generated thoughts: {thoughts}")
                strategy = result.strategy
                
                # Add initial thoughts to the graph
                for i, thought_content in enumerate(thoughts):
                    # Skip if the thought is just a single character or empty
                    if len(thought_content.strip()) <= 1:
                        print(f"[DEBUG] Skipping single character thought: '{thought_content}'")
                        continue
                        
                    thought_id = f"{layer}_{i}"
                    G.add_node(thought_id, data=Thought(
                        id=thought_id,
                        content=thought_content,
                        layer=layer,
                        strategy=strategy
                    ))
                    print(f"[DEBUG] Added thought {thought_id}: {thought_content[:50]}...")
                    
            elif layer == 0:
                # Generate initial thoughts for a nested graph
                parent_context = self._format_graph_for_context(parent_graph)
                
                enhanced_query = f"""
                IMPORTANT: Please process this query in its original language.
                DO NOT split the query into individual characters.
                Generate {self.max_nodes} complete, meaningful thoughts related to this query.
                Query: {query}
                Context: {parent_context}
                """
                
                print(f"\n[DEBUG] Depth {depth}, Layer {layer}: Generating nested initial thoughts for query: {query}")
                result = self.t0(enhanced_query, self.max_nodes)
                
                # Extract and validate thoughts
                thoughts = result.thoughts
                if not thoughts or isinstance(thoughts, str):
                    if isinstance(thoughts, str):
                        import re
                        thoughts = re.split(r'\d+\.\s+', thoughts)
                        thoughts = [t.strip() for t in thoughts if t.strip()]
                    
                    if not thoughts:
                        thoughts = [
                            f"Decompose {query} into smaller components",
                            f"Identify key challenges in {query}",
                            f"Outline approach for addressing {query}"
                        ]
                
                print(f"[DEBUG] Generated nested thoughts: {thoughts}")
                strategy = result.strategy
                
                # Add initial thoughts to the graph
                for i, thought_content in enumerate(thoughts):
                    # Skip if the thought is too short
                    if len(thought_content.strip()) <= 1:
                        print(f"[DEBUG] Skipping single character thought: '{thought_content}'")
                        continue
                        
                    thought_id = f"{layer}_{i}"
                    G.add_node(thought_id, data=Thought(
                        id=thought_id,
                        content=thought_content,
                        layer=layer,
                        strategy=strategy
                    ))
                    print(f"[DEBUG] Added nested thought {thought_id}: {thought_content[:50]}...")
            else:
                # Generate subsequent thoughts
                graph_context = self._format_graph_for_context(G)
                
                enhanced_query = f"""
                IMPORTANT: Please process this query in its original language.
                DO NOT split the query into individual characters.
                Generate {self.max_nodes} complete, meaningful thoughts that build on existing thoughts.
                
                Query: {query}
                
                Current thoughts: {graph_context}
                
                When providing edges, please use this format exactly: [(source_id, target_id), (source_id, target_id)]
                For example: [("0_0", "1_0"), ("0_1", "1_1")]
                """
                
                print(f"\n[DEBUG] Depth {depth}, Layer {layer}: Generating subsequent thoughts")
                result = self.te(enhanced_query, graph_context, self.max_nodes)
                
                # Extract and validate new thoughts
                new_thoughts = result.new_thoughts
                new_edges = result.new_edges
                if not new_thoughts or isinstance(new_thoughts, str):
                    # Fallback if the output format is incorrect
                    if isinstance(new_thoughts, str):
                        # Try to parse if it's a string
                        import re
                        new_thoughts = re.split(r'\d+\.\s+', new_thoughts)
                        new_thoughts = [t.strip() for t in new_thoughts if t.strip()]
                    
                    # If still empty, create default thoughts based on existing nodes
                    if not new_thoughts:
                        prev_thoughts = [data['data'].content for _, data in G.nodes(data=True)]
                        new_thoughts = [
                            f"Explore implications of {prev_thoughts[0] if prev_thoughts else query}",
                            f"Consider alternative perspectives on {prev_thoughts[-1] if prev_thoughts else query}",
                            f"Synthesize findings from previous thoughts"
                        ]
                
                print(f"[DEBUG] Generated subsequent thoughts: {new_thoughts}")
                print(f"[DEBUG] New edges: {new_edges}")
                
                # Ensure new_edges is in the correct format
                if new_edges:
                    # If new_edges is a string, try to parse it
                    if isinstance(new_edges, str):
                        try:
                            import ast
                            parsed_edges = ast.literal_eval(new_edges)
                            if isinstance(parsed_edges, list):
                                new_edges = parsed_edges
                            else:
                                new_edges = []
                        except:
                            # If parsing fails, set to empty list
                            print("[DEBUG] Error parsing edges, using empty list")
                            new_edges = []
                else:
                    new_edges = []
                
                # Make sure new_edges is a list of tuples
                if not all(isinstance(edge, tuple) and len(edge) == 2 for edge in new_edges):
                    print("[DEBUG] Edges not in correct format, creating default edges")
                    # Create default edges connecting previous layer to this one
                    new_edges = []
                    prev_layer_nodes = [n for n, data in G.nodes(data=True) 
                                      if data['data'].layer == layer-1]
                    
                    for prev_node in prev_layer_nodes:
                        for i in range(len(new_thoughts)):
                            if len(new_thoughts[i].strip()) > 1:  # Skip single character thoughts
                                new_edges.append((prev_node, f"{layer}_{i}"))
                
                strategy = result.strategy
                
                # Add new thoughts to the graph
                for i, thought_content in enumerate(new_thoughts):
                    # Skip if the thought is just a single character or empty
                    if len(thought_content.strip()) <= 1:
                        print(f"[DEBUG] Skipping single character thought: '{thought_content}'")
                        continue
                        
                    thought_id = f"{layer}_{i}"
                    G.add_node(thought_id, data=Thought(
                        id=thought_id,
                        content=thought_content,
                        layer=layer,
                        strategy=strategy
                    ))
                    print(f"[DEBUG] Added subsequent thought {thought_id}: {thought_content[:50]}...")
                
                # Add edges
                print(f"[DEBUG] Adding edges: {new_edges}")
                for edge in new_edges:
                    try:
                        source, target = edge
                        if source in G and target in G:
                            if self._is_edge_safe(G, source, target):
                                print(f"[DEBUG] Adding edge... {source} → {target}")
                                G.add_edge(source, target)
                                print(f"[DEBUG] Added edge: {source} → {target}")
                            else:
                                print(f"[DEBUG] Edge not added, cycle found: {new_edges}")
                        else:
                            print(f"[DEBUG] Edge not added, nodes do not exist: {source} → {target}")
                    except ValueError as e:
                        print(f"[DEBUG] Error unpacking edge {edge}: {e}")
                        # Continue with other edges
            
            # Check if any thought in this layer is final
            for node_id, data in list(G.nodes(data=True)):
                thought = data['data']
                if thought.layer == layer:
                    # Check if this thought is complex
                    graph_context = self._format_graph_for_context(G)
                    
                    print(f"[DEBUG] Depth {depth}, Layer {layer}: Checking complexity of thought {node_id}: {thought.content[:50]}...")
                    is_complex, reason = self.c(thought.content, graph_context)
                    print(f"[DEBUG] Complexity check result: {is_complex}, Reason: {reason}")
                    
                    if is_complex and depth < self.max_depth:
                        # Mark as complex
                        thought.is_complex = True
                        print(f"[DEBUG] Thought {node_id} is complex, starting recursive processing")
                        
                        # Recursively process this thought
                        nested_heritage = heritage + [(layer, int(node_id.split('_')[1]))]
                        answer, nested_graph = self._recursive_agot(
                            thought.content, 
                            nested_heritage, 
                            G, 
                            depth + 1
                        )
                        
                        # Store the result
                        thought.answer = answer
                        thought.nested_graph = nested_graph
                        print(f"[DEBUG] Completed recursive processing for thought {node_id}, answer: {answer[:50]}...")
                        
                    else:
                        # Evaluate the thought directly
                        thought.is_complex = False
                        print(f"[DEBUG] Evaluating thought {node_id} directly")
                        thought.answer = self.eval(thought.content, graph_context)
                        print(f"[DEBUG] Thought {node_id} evaluation result: {thought.answer[:50]}...")
            
            # Check if we need to add another layer
            if layer == self.max_layers - 1 or self._should_terminate(G):
                break
        
        # Synthesize final answer from this graph
        graph_context = self._format_graph_for_context(G)
        print(f"\n[DEBUG] Depth {depth}: Synthesizing final answer")
        final_answer = self.phi(query, graph_context)
        print(f"[DEBUG] Final answer: {final_answer[:100]}...")
        
        return final_answer, G
    
    def _should_terminate(self, G):
        """Check if we should terminate early based on the graph state."""
        # This is a simple implementation - could be made more sophisticated
        # For example, check if a high-quality answer has already been found
        return False
    
    def forward(self, query):
        """Run the AGoT algorithm on the query."""
        final_answer, graph = self._recursive_agot(query, [])
        return {
            "answer": final_answer,
            "graph": graph
        }
    
    def visualize_graph(self, graph, figsize=(12, 8)):
        """Visualize the graph of thoughts."""
        plt.figure(figsize=figsize)
        
        # Create position layout
        pos = {}
        layer_counts = {}
        
        for node_id, data in graph.nodes(data=True):
            thought = data['data']
            layer = thought.layer
            
            if layer not in layer_counts:
                layer_counts[layer] = 0
            
            # Position nodes in layers
            pos[node_id] = (layer, -layer_counts[layer])
            layer_counts[layer] += 1
        
        # Draw nodes
        node_colors = []
        for node_id, data in graph.nodes(data=True):
            thought = data['data']
            if thought.is_complex:
                node_colors.append('orange')
            else:
                node_colors.append('skyblue')
        
        nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=700, alpha=0.8)
        nx.draw_networkx_edges(graph, pos, width=1.0, alpha=0.5, arrows=True)
        
        # Draw labels
        labels = {}
        for node_id, data in graph.nodes(data=True):
            thought = data['data']
            labels[node_id] = f"{node_id}\n{thought.content[:20]}..."
        
        nx.draw_networkx_labels(graph, pos, labels, font_size=8)
        
        plt.title("Adaptive Graph of Thoughts")
        plt.axis('off')
        plt.tight_layout()
        plt.show()


In [ ]:
import dspy
import os
lm = dspy.LM('gpt-4o-mini', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)
agot = AdaptiveGraphOfThoughts(max_depth=1, max_layers=3, max_nodes=3)
# Run on a complex query
result = agot(query="What's the best opening move in Catan? Explain in high detail 3 potential scenarios where it showcases the best performance.")

# Get the answer and graph
answer = result["answer"]
graph = result["graph"]

print(f"Final Answer: {answer}")
print("\nGenerated Thoughts:")
for node_id, data in graph.nodes(data=True):
    thought = data['data']
    print(f"- {thought}")
# Visualize the graph
# agot.visualize_graph(graph)



[DEBUG] Depth 0, Layer 0: Generating initial thoughts for query: What's the best opening move in Catan? Explain in high detail 3 potential scenarios where it showcases the best performance.
[DEBUG] Generated thoughts: ['One effective opening move in Catan is to place a settlement on a hex that has access to both brick and wood, ideally on a location with a high probability number (like 6 or 8). This positioning allows for quick access to roads and settlements, facilitating expansion towards key resources like wheat and ore. In a scenario where the player is able to build a road towards a port, this can lead to a strong trading position, especially if the player can monopolize a resource that is scarce for others.', 'Another strong opening move is to place a settlement on a location that provides access to wheat, sheep, and ore. This combination is crucial for building cities and development cards, which can be game-changers. In a scenario where the player is able to build a city early

In [5]:
result

{'answer': "One of the best opening moves in Catan is to place a settlement at the intersection of high-yield resources, particularly wheat, ore, and wood. This strategy can be illustrated through three potential scenarios:\n\n1. **Scenario 1: Balanced Resource Access**  \n   By placing a settlement on a location that provides access to wheat, wood, and brick, a player can ensure a steady supply of essential resources for building roads and settlements. This balanced access allows for quick expansion, enabling the player to secure additional settlements and potentially block opponents from key areas. The early game advantage of having multiple settlements can lead to a strong mid-game position.\n\n2. **Scenario 2: Port Access**  \n   Establishing a settlement near a port, particularly a 2:1 port for a resource that the player has in abundance, can be a game-changer. For instance, if a player has access to a lot of wood and places a settlement next to a wood port, they can trade wood fo

In [26]:
from IPython.display import display, Markdown

display(Markdown(result['answer']))

One of the best opening moves in Catan is to place a settlement at the intersection of high-yield resources, particularly wheat, ore, and wood. This strategy can be illustrated through three potential scenarios:

1. **Scenario 1: Balanced Resource Access**  
   By placing a settlement on a location that provides access to wheat, wood, and brick, a player can ensure a steady supply of essential resources for building roads and settlements. This balanced access allows for quick expansion, enabling the player to secure additional settlements and potentially block opponents from key areas. The early game advantage of having multiple settlements can lead to a strong mid-game position.

2. **Scenario 2: Port Access**  
   Establishing a settlement near a port, particularly a 2:1 port for a resource that the player has in abundance, can be a game-changer. For instance, if a player has access to a lot of wood and places a settlement next to a wood port, they can trade wood for other resources more efficiently. This strategy can lead to a resource surplus, allowing the player to build more settlements and roads than opponents, ultimately leading to victory points.

3. **Scenario 3: High-Yield Resource Focus**  
   Prioritizing a settlement on high-yield resource tiles, such as a wheat tile with a high number (like 6 or 8), can significantly enhance a player's resource generation. This move can lead to rapid development, as wheat is crucial for building settlements and cities. If the player can secure this position early, they can outpace opponents in development cards and cities, which are vital for scoring points and gaining advantages in the game.

In summary, the best opening move in Catan often involves strategic placement that maximizes resource access and potential for expansion, setting the stage for a strong game performance.

In [27]:
agot = AdaptiveGraphOfThoughts(max_depth=1, max_layers=2, max_nodes=2)
# Run on a complex query
result = agot(query="What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.")

# Get the answer and graph
answer = result["answer"]
graph = result["graph"] 


[DEBUG] Depth 0, Layer 0: Generating initial thoughts for query: What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.
[DEBUG] Generated thoughts: ["One potential niche for a Generative AI product could be personalized content creation for social media influencers. This could involve developing a tool that generates tailored captions, hashtags, or even visual content based on the influencer's style and audience engagement metrics. To build in public, you could start by sharing your development process on platforms like Twitter or LinkedIn, engaging with influencers to gather insights, and iterating based on their feedback.", "Another niche could be AI-driven educational tools that create customized learning materials for students based on their individual learning styles and progress. This could include generating

In [10]:
markdown_content = result['answer']

display(Markdown(markdown_content))

To build a digital product using Generative AI, consider these five niches:

1. **Personalized Marketing Content Creation**: Develop a tool that generates tailored marketing materials based on user data and preferences. 
2. **AI-Driven Educational Tools**: Create an application that customizes learning experiences and materials for students based on their individual learning styles and progress.
3. **AI-Assisted Mental Health Applications**: Build a platform that offers personalized mental health support and resources, utilizing AI to adapt to user needs.
4. **Generative AI for Creative Writing**: Design a tool that assists writers by generating prompts, story ideas, or even full drafts based on user input.
5. **AI-Powered Design Tools**: Create a product that helps users generate design concepts or graphics based on their specifications and preferences.

To build in public and test your hypotheses, follow these actions:

- **Share Your Journey**: Document your process on social media platforms like Twitter or LinkedIn. Share updates, challenges, and milestones to engage your audience.
- **Create a Landing Page**: Develop a simple website to showcase your product idea, collect emails, and build a community interested in your product.
- **Engage with Your Audience**: Use platforms like Discord or Reddit to create a community around your niche. Encourage feedback and discussions about your product.
- **Iterate Based on Feedback**: Regularly seek input from your audience and make adjustments to your product based on their suggestions and needs.
- **Launch a Minimum Viable Product (MVP)**: Once you have a prototype, release it to your audience for testing. Use their feedback to refine and improve your product.

By following these steps, you can effectively build a digital product in the Generative AI space while engaging with your audience and validating your ideas.

In [28]:
agot = AdaptiveGraphOfThoughts(max_depth=1, max_layers=3, max_nodes=3)
# Run on a complex query
result = agot(query="What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.")

# Get the answer and graph
answer = result["answer"]
graph = result["graph"] 


[DEBUG] Depth 0, Layer 0: Generating initial thoughts for query: What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.
[DEBUG] Generated thoughts: ['One potential niche for a Generative AI product could be personalized content creation for social media influencers. This could involve developing a tool that generates tailored posts, captions, or even visual content based on user preferences and trends. To build in public, you could share your development process on platforms like Twitter or LinkedIn, engaging with influencers to gather feedback and iterate on your product.', 'Another niche could be AI-driven educational tools that create customized learning materials for students. This could include generating quizzes, study guides, or interactive lessons based on individual learning styles. Building in public coul

In [12]:
markdown_content = result['answer']

display(Markdown(markdown_content))

To build a digital product using Generative AI, consider the following five niches:

1. **AI-driven Educational Tools**: Develop tools that personalize learning experiences using AI to adapt content to individual student needs.
2. **Personalized Marketing Solutions**: Create AI systems that analyze customer data to generate tailored marketing strategies and content.
3. **Mental Health Support**: Build applications that provide AI-driven mental health resources, such as chatbots for therapy or mood tracking.
4. **Virtual Event Planning**: Design platforms that utilize AI to streamline the planning and execution of virtual events, enhancing user engagement.
5. **AI-assisted Content Creation**: Focus on tools that help creators generate content efficiently, such as writing assistants or design generators.

To build in public and test your hypotheses, follow these actions:

- **Share Your Journey**: Document your process on social media platforms like Twitter or LinkedIn. Share updates, challenges, and milestones.
- **Engage with Your Audience**: Create polls or ask for feedback on your ideas to involve your audience in the development process.
- **Build a Community**: Start a newsletter or a Discord channel where you can share insights and gather a group of interested users.
- **Iterate Based on Feedback**: Use the feedback from your audience to refine your product and make adjustments based on their needs.
- **Launch Early**: Consider launching a minimum viable product (MVP) to gather real-world usage data and further engage your audience.

By following these steps, you can effectively build a digital product while fostering an engaged community around your Generative AI ideas.

In [30]:
agot = AdaptiveGraphOfThoughts(max_depth=2, max_layers=5, max_nodes=5)
# Run on a complex query
result = agot(query="What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.")

# Get the answer and graph
answer = result["answer"]
graph = result["graph"] 


[DEBUG] Depth 0, Layer 0: Generating initial thoughts for query: What is the best way of building a digital product? Help me finding 5 niches where I could build a small Generative AI product. Describe the actions I must do to build in public and build an audience to test out my hyphothesis.
[DEBUG] Generated thoughts: ['**Niche: Personalized Content Creation** - Develop a Generative AI tool that helps users create personalized content, such as blog posts, social media updates, or marketing materials. Actions include researching user needs, creating a prototype, and sharing progress on social media to gather feedback.', '**Niche: AI-Powered Art Generation** - Create a platform that allows users to generate unique artwork using AI algorithms. To build in public, document the creative process through videos or live streams, and engage with artists and art enthusiasts on platforms like Instagram or Twitter.', '**Niche: Automated Resume and Cover Letter Builder** - Design a Generative AI 

KeyboardInterrupt: 

In [16]:
markdown_content = result['answer']

display(Markdown(markdown_content))

To build a digital product, particularly a small Generative AI product, consider the following five niches:

1. **Personalized Learning Platforms**: Create AI-driven educational tools that adapt to individual learning styles. Actions to build in public include sharing your development process on social media, conducting surveys to understand user needs, and offering beta access to early adopters.

2. **Content Creation Assistants**: Develop tools that help users generate written content, such as blog posts or marketing copy. Engage with potential users by sharing tips on content creation, hosting webinars, and soliciting feedback on your product iterations.

3. **AI Art Generators**: Focus on creating platforms that allow users to generate unique artwork using AI. Build an audience by showcasing generated art on platforms like Instagram, running contests, and collaborating with artists to promote your tool.

4. **Virtual Health Assistants**: Create AI tools that provide personalized health advice or mental wellness support. To build in public, share insights on health trends, collaborate with health professionals, and create a community around wellness discussions.

5. **Automated Customer Support Solutions**: Develop AI chatbots that can handle customer inquiries. Build an audience by sharing case studies of successful implementations, offering free trials, and engaging in discussions about customer service improvements.

To effectively build in public and test your hypotheses, consistently share your progress, gather user feedback, and iterate based on the insights you receive. Utilize social media, blogs, and community forums to engage with your audience and refine your product based on their needs.

In [22]:
# needs work
thoughts_table = "\n".join([f"| {node_id} | {data['data'].layer} | {data['data'].answer} | {data['data'].content} | {data['data'].strategy}" for node_id, data in graph.nodes(data=True)])
display(Markdown(thoughts_table))
    

| 0_0 | 0 | A potential niche for a Generative AI product is personalized content creation for social media influencers. This tool could generate tailored captions, hashtags, and visual content based on the influencer's unique style and audience engagement metrics. To effectively build this product in public, you should share your development process on platforms like Twitter or LinkedIn, actively engage with influencers to gather their insights, and iterate on the tool based on their feedback. | One potential niche for a Generative AI product could be personalized content creation for social media influencers. This could involve developing a tool that generates tailored captions, hashtags, or even visual content based on the influencer's style and audience engagement metrics. To build in public, you could start by sharing your development process on platforms like Twitter or LinkedIn, engaging with influencers to gather insights, and iterating based on their feedback. | The strategy to explore these thoughts involves conducting market research to validate the identified niches, engaging with potential users through social media and forums to gather insights, and iterating on the product based on feedback. Additionally, documenting the development journey and sharing updates regularly will help in building an audience and fostering a community around the product.
| 0_1 | 0 | To build an audience for AI-driven educational tools that create customized learning materials, collaborate with educators and students, share prototypes on educational forums, and host webinars to demonstrate the product's capabilities while collecting user feedback. This approach not only showcases the tool's effectiveness but also engages potential users in the development process. | Another niche could be AI-driven educational tools that create customized learning materials for students based on their individual learning styles and progress. This could include generating quizzes, study guides, or interactive lessons. To build an audience, you could collaborate with educators and students, share your prototypes on educational forums, and host webinars to demonstrate the product's capabilities while collecting user feedback. | The strategy to explore these thoughts involves conducting market research to validate the identified niches, engaging with potential users through social media and forums to gather insights, and iterating on the product based on feedback. Additionally, documenting the development journey and sharing updates regularly will help in building an audience and fostering a community around the product.